# Projetando um app de controle de finanças

A ideia é construir uma aplicação que ajude a ver para onde o dinheiro está indo.

* Definição de metas;
* Cadastro de contas;
* Cadastro de cartões; 

### Models

In [11]:
import enum
import os

import datetime

from sqlalchemy import (Column, DateTime, Enum, ForeignKey, Integer, Numeric,
                        String, Table, create_engine, select)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, scoped_session, sessionmaker



In [3]:
Base = declarative_base()


class TransactionType(enum.Enum):
    """Enum."""

    INCOME = 0
    OUTGO = 1
    TRANSFER = 2


class PaymentType(enum.Enum):
    
    CASH = 0
    DEBIT = 1
    CREDIT = 2


class MySession():
    """Sessão com BD.
    Para definir a sessão com o BD na aplicação. Para os
    testes, passando o parâmetro test=True, um BD na memória
    """

    def __init__(self, base, test=False):
        """Inicializa."""
        if test:
            path = ':memory:'
        else:
            path = ':memory:'
        self._engine = create_engine('sqlite:///' + path, convert_unicode=True)
        Session = sessionmaker(bind=self._engine)
        if test:
            self._session = Session()
        else:
            self._session = scoped_session(Session)
            base.metadata.bind = self._engine

    @property
    def session(self):
        """Session."""
        return self._session

    @property
    def engine(self):
        """Engine."""
        return self._engine
    
    
class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True)
    password = Column(String(30))
    
    def __init__(self, username, password):
        self.username = username
        self.password = password


account_transaction = Table('Account_Transaction', Base.metadata,
                           Column('account_id', Integer, ForeignKey('account.id')),
                           Column('transaction_id', Integer, ForeignKey('transaction.id')))

class Account(Base):
    """docstring for Account"""
    __tablename__ = 'account'
    id = Column(Integer, primary_key=True)
    bank = Column(String(50))
    description = Column(String(50), unique=True)
    balance = Column(Numeric(asdecimal=False))
    user_id = Column(Integer, ForeignKey('user.id'))
    creditcard_id = Column(Integer, ForeignKey(
        "creditcard.id"))
    transaction = relationship('Account',
                              secondary=account_transaction,
                              back_populates='transaction')

    def __init__(self, description, bank, balance, user, creditcard = None):
        self.description = description
        self.bank = bank
        self.balance = balance
        self.user_id = user.id
        if(creditcard):
            self.creditcard_id = creditcard.id


class Transaction(Base):
    """docstring for Transaction"""
    __tablename__ = 'transaction'
    id = Column(Integer, primary_key=True)
    description = Column(String(50))
    outgo_id = Column(Integer, ForeignKey('outgo.id'))
    income_id = Column(Integer, ForeignKey('income.id'))
    account = relationship('Transaction',
                          secondary=account_transaction,
                          back_populates='account')
    
    def __init__(self, outgo, income):
        self.outgo_id = outgo.id
        self.income_id = income.id


class Category(Base):
    """docstring for Categoria"""
    __tablename__ = 'category'
    id = Column(Integer, primary_key=True)
    description = Column(String(50), unique=True)

    def __init__(self, description):
        self.description = description


class Income(Base):
    """docstring for Income"""
    __tablename__ = 'income'
    id = Column(Integer, primary_key=True)
    value = Column(Numeric(asdecimal=False))
    description = Column(String(50))
    date = Column(DateTime)
    category_id = Column(Integer, ForeignKey('category.id'))

    def __init__(self, value, description, date, category):
        super(Income, self).__init__()
        self.value = value
        self.description = description
        self.date = date
        self.category_id = category.id


class Outgo(Base):
    """docstring for Outgo"""
    __tablename__ = 'outgo'
    id = Column(Integer, primary_key=True)
    value = Column(Numeric(asdecimal=False))
    description = Column(String(50))
    date = Column(DateTime)
    category_id = Column(Integer, ForeignKey('category.id'))

    def __init__(self, value, description, date, category):
        super(Outgo, self).__init__()
        self.value = value
        self.description = description
        self.date = date
        self.category_id = category.id


creditcard_purchase = Table('creditcard_purchase', Base.metadata, 
                            Column('creditcard_id', Integer, 
                                   ForeignKey('creditcard.id')), 
                            Column('purchase_id', Integer, 
                                   ForeignKey('purchase.id')))


class CreditCard(Base):
    __tablename__ = 'creditcard'
    id = Column(Integer, primary_key=True)
    limit = Column(Numeric(asdecimal=False))
    date_pay = Column(DateTime)
    account_id = Column(Integer, ForeignKey('account.id'))
    purchase = relationship('Purchase', secondary=creditcard_purchase, back_populates='creditcard')
    
    def __init__(self, limit, account, date_pay):
        self.limit = limit
        self.account_id = account.id
        self.date_pay = date_pay
        

class Purchase(Base):
    __tablename__ = 'purchase'
    id = Column(Integer, primary_key=True)
    description = Column(String(50))
    value = Column(Numeric(asdecimal=True))
    date = Column(DateTime)
    payed_with = Column(Enum(PaymentType))
    category_id = Column(Integer, ForeignKey('category.id'))
    creditcard = relationship('CreditCard', 
                              secondary=creditcard_purchase, 
                              back_populates='purchase')
    
    def __init__(self, description, value, date, payed_with):
        self.description = description
        self.value = value
        self.date = date
        self.payed_with = payed_with



In [4]:
mysession = MySession(Base)
dbsession = mysession.session
engine = mysession.engine

In [5]:
Base.metadata.create_all()

In [ ]:
user1 = dbsession.query(User).first()

In [ ]:
vars(user1)

In [12]:
def user_registration(username:str, password:str):
    user = User(username, password)
    dbsession.add(user)
    dbsession.commit()

def create_account(bank:str, description:str, balance:str, user:User):
    account = Account(description, bank, balance, user)
    dbsession.add(account)
    dbsession.commit()

def add_creditcard(limit:float, account:Account, date_pay:datetime):
    creditcard = CreditCard(limit, account, date_pay)
    dbsession.add(creditcard)
    dbsession.commit()

In [7]:
user_registration('gi', 'senha')

usern = dbsession.query(User).filter(User.username == 'gi').first()

create_account('Itaú', 'Conta', 500, usern)

accountn = dbsession.query(Account).all()

In [17]:
acc = dbsession.query(Account).join(User).filter(User.id == Account.user_id).first()
add_creditcard(1000, acc, datetime.datetime.now())

In [16]:
datetime.datetime.now()

datetime.datetime(2019, 5, 15, 20, 54, 26, 411450)

In [21]:
vars(dbsession.query(Account).join(CreditCard).filter(CreditCard.id == Account.creditcard_id).first())

AmbiguousForeignKeysError: Can't determine join between 'account' and 'creditcard'; tables have more than one foreign key constraint relationship between them. Please specify the 'onclause' of this join explicitly.

In [9]:
vars(dbsession.query(Account).join(User).filter(User.id == Account.user_id).first())

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f3b7b0f1dd8>,
 'description': 'Conta',
 'id': 1,
 'creditcard_id': None,
 'balance': 500,
 'bank': 'Itaú',
 'user_id': 1}

In [10]:
vars(dbsession.query(Account).filter(Account.user_id == 1).first())

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f3b7b0f1dd8>,
 'description': 'Conta',
 'id': 1,
 'creditcard_id': None,
 'balance': 500,
 'bank': 'Itaú',
 'user_id': 1}

In [ ]:
#----------------------------
# Turn Foreign Key Constraints ON for
# each connection.
#----------------------------

from sqlalchemy.engine import Engine
from sqlalchemy import event

@event.listens_for(Engine, "connect")
def set_sqlite_pragma(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
    cursor.close()

#----------------------------
# Create the engine
#----------------------------

from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

#----------------------------
# Create the Schema
#----------------------------

Base.metadata.create_all(engine)

#----------------------------
# Create the Session class 
#----------------------------

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

#----------------------------
# Populate the database 
#----------------------------

'''tag_cool = Tag(name='cool')
tag_car = Tag(name='car')
tag_animal = Tag(name='animal')

comment_rhino = Comment(text='Rhinoceros, often abbreviated as rhino, is a group of five extant species of odd-toed ungulates in the family Rhinocerotidae.')

image_car = Image(uuid='uuid_car', \
    tags=[tag_car, tag_cool], \
    created_at=(datetime.utcnow() - timedelta(days=1)))

image_another_car = Image(uuid='uuid_anothercar', \
    tags=[tag_car])

image_rhino = Image(uuid='uuid_rhino', \
    tags=[tag_animal], \
    comments=[comment_rhino])
'''



In [ ]:
user = User('adauto', 'senha')


In [ ]:
creditcard = CreditCard()

In [ ]:
account = Account("Conta Corrente", "Nubank", 500, user)

In [ ]:
session = Session()
user = User('adauto', 'senha')
session.add(user)

account = Account("Conta Corrente", "Nubank", 500, user)
session.add(account)

# Commit the changes:
session.commit() 


In [ ]:
accounts = session.query(Account).filter(Account.id == 1).first()


In [ ]:
accounts.user_id = 1

In [ ]:
session.add(accounts)
session.commit()

In [ ]:
vars(accounts)
session.close()

In [ ]:

def create_account(balance = 0, credit_card = None):
    return {'balance': balance, 'credit_card': credit_card}

def purchase_with_credit(description:str,value:int, date_acquisition:str, parcels=1, finished=False):
    return {'value': value, 'description': description, \
            'date_acquisition': date_acquisition, 'parcels': parcels, 'finished': finished}

def purchase_with_debit(value):
    pass

def create_credit_card(limit = 0, purchases = set()):
    return {'limit': limit, 'purchases': purchases}

credit_card = create_credit_card(limit=6500)
account = create_account(210, credit_card)

bank = {'name': 'Nubank', 'account': account}

bank['account']['credit_card']['purchases'] = [purchase_with_credit('compra', 10, '10/02/2019')]

bank

In [ ]:
# Create a new Session and add the data:
session = Session()

session.add(tag_cool)
session.add(tag_car)
session.add(tag_animal)

session.add(comment_rhino)

session.add(image_car)
session.add(image_another_car)
session.add(image_rhino)

# Commit the changes:
session.commit() 
